## Search for data on AWS

In [ ]:
#https://github.com/sat-utils/sat-search
from satsearch import Search
from satstac import Items
import json
import geopandas as gpd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Add an area of interest polygon to search
#geojson.io
with open('wa-bbox.json') as f:
    aoi = json.loads(f.read())

In [ ]:
# Custom syntax (additional fields, query strings instead of query dict)
properties =  ["eo:row=027",
               "eo:column=047",
               "landsat:tier=T1"] 
results = Search.search(collection='landsat-8-l1', 
                        intersects=aoi, 
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % results.found())

In [ ]:
# Save for later/sharing with others
items = results.items()
items.save('items.json')
items = Items.load('items.json')
#items.bbox()

In [ ]:
# Load results to pandas geodataframe
gf = gpd.read_file('items.json')
gf = gf.sort_values('datetime').reset_index(drop=True)
print('records:', len(gf))
gf.head()

In [ ]:
# Plot search AOI and frames on a map
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import geoviews as gv

footprints = gf.loc[:,('id','geometry')].hvplot(geo=True)
tiles = gv.tile_sources.CartoEco.options(width=700, height=500).redim.range(Latitude=(45, 50), Longitude=(-126,-120)) 
labels = gv.tile_sources.StamenLabels.options(level='annotation')
tiles * footprints * labels

In [ ]:
# Alternatively see https://landsat.stac.cloud/item/kcPK8Xt2XpdjqL3FVBDhJz2EjeXsYtz7xh/5vNQJWyXHzm2VzS4b4KaeT4w9fWu5Qnp8uqqnS77/ZCMYCwtTqUMQ5JXpM4aiuM2qaDboboHpy3Dc1KzTSukAh/zBTMeeQQWjwvEEJZpQpk4TuF6nAUc5gUUXcmZyxchWxbYg2VJw6gRVD5cJbcr9hx3Wp8vVu28gPDc3o?si=0&t=thumbnail#8/47.458510/-123.386500
from ipywidgets import interact
from IPython.display import display, Image

def browse_images(items):
    n = len(items)

    def view_image(i=0):
        item = items[i]
        print(f"id={item.id}\tdate={item.datetime}\tcloud%={item['eo:cloud_cover']}")
        display(Image(item.asset('thumbnail')['href']))
    
    interact(view_image, i=(0,n-1))
    


In [ ]:
browse_images(items)

# Plotting and analysis

In [ ]:
# Load into xarray and do computations with dask
# Select 10 'workers' under 'manual scaling' menu below and click 'Scale'
# Click on the 'Dashboard link' to monitor calculation progress
import dask
from dask_kubernetes import KubeCluster
from dask.distributed import Client
from dask.distributed import wait, progress

cluster = KubeCluster(n_workers=10)
cluster

In [ ]:
# Attach Dask to the cluster
client = Client(cluster)

In [ ]:
# Import required libraries
import rasterio
import xarray as xr
import hvplot.xarray
import hvplot.pandas

In [ ]:
env = rasterio.Env(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                  CPL_VSIL_CURL_USE_HEAD=False,
                  CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
                 )

In [ ]:
# Plot single band, full resolution
item = items[5]
print(item.assets.keys())
print(item.assets_by_common_name.keys())

with env:
    with rasterio.open(item.asset('red')['href']) as src:
        width = src.width
        blockx = src.profile['blockxsize']
        blocky = src.profile['blockysize']
        #print(src.profile)
        xchunk = int(width/blockx)*blockx
        ychunk = blocky
        da = xr.open_rasterio(src, chunks={'band': 1, 'x': xchunk, 'y': ychunk})
da

In [ ]:
print(item.date)
img = da.hvplot.image(rasterize=True, width=700, height=500, cmap='magma')
img

In [ ]:
# Display with reprojection to lat/lon
print(item.date)
from cartopy import crs as ccrs
crs = ccrs.UTM(zone='10n') #hvplot needs to know original CRS
img = da.hvplot.image(crs=crs,rasterize=True, width=700, height=500, cmap='magma')
img

In [ ]:
# Get single image
import intake
cat = intake.Catalog('intake-landsat-aws.yaml')
ds = cat.aws_landsat_8_singleband(url=item.asset('B1')['href']).to_dask()
ds

In [ ]:
# All 30m bands for particular item
row = item["eo:row"]
path = item["eo:column"]
pid = item['landsat:product_id']
ds = cat.aws_landsat_8_30m(path=path, row=row, pid=pid).to_dask()
ds

In [ ]:
print(item.date)
img = ds.hvplot.image('x', 'y', groupby='band', rasterize=True, width=700, height=500, cmap='magma')
img

In [ ]:
# A bigger calculation - all ndvi
# option 1) download only NIR and Red bands locally
#redtifs = items.download('red', path='red/${date}')
#nirtifs = items.download('nir', path='nir/${date}')

In [ ]:
def create_ndvi_dataset(item, chunks={'band': 1, 'x': xchunk, 'y': ychunk}):
    '''A function to load multiple landsat bands into an xarray dataset'''
    row = item["eo:row"]
    path = item["eo:column"]
    pid = item['landsat:product_id']
    ds = cat.aws_landsat_8_ndvi(path=path, row=row, pid=pid).to_dask()
    
    return ds

In [ ]:
# Merge all acquisitions into a single large Dataset, 
# this may take a minute b/c running sequentially on local machine
from ipywidgets import IntProgress
from IPython.display import display
probar = IntProgress(value=0, min=0, max=len(items), step=1, 
                     description='Loading:')
display(probar)

datasets = []
for item in items:
    probar.value += 1
    try:
        #print('loading...', item.date)
        ds = create_ndvi_dataset(item)
        datasets.append(ds)
    except Exception as e:
        print('ERROR loading, skipping acquistion!')
        print(e)

In [ ]:
# Create an xarray data array
# This takes a while to expand dimensions
index = gpd.pd.DatetimeIndex(items.dates(), name='time')
da = xr.concat(datasets, dim=index)
print('Dataset size (Gb): ', da.nbytes/1e9)
da

In [ ]:
# Can rearrange so that each band is a 'data variable' in a dataset
DS = da.to_dataset(dim='band')
DS

In [ ]:
# Can adjust dask chunks if desired:
# (they get automatically adjusted when concatenating)
xchunk = DS.dims['x']
chunks = {'x': xchunk, 'y': ychunk}
DS = DS.chunk(chunks)
DS

In [ ]:
# Nothing actually computed yet
NDVI = (DS[5] - DS[4]) / (DS[5] + DS[4])
NDVI

In [ ]:
# Make an interactive plot that records clicked points
taps = []
def record_coords(x, y):
    if None not in [x,y]:
        taps.append([x, y])
    return hv.Points(taps)

In [ ]:
# NOTE: this will take a minute to load and is best viewed on a wide monitor
# the time slider can get hidden on small screens
img = NDVI.hvplot('x', 'y', groupby='time', dynamic=True, rasterize=True, width=700, height=500, cmap='magma')
tap = hv.streams.SingleTap(transient=True, source=img)
clicked_points = hv.DynamicMap(record_coords, streams=[tap])

img * clicked_points.options(size=10, color='w')

In [ ]:
# Points clicked are stored in the 'taps list'
if len(taps) == 0:
    taps = [(562370, 5312519)]

print('Selected points:')
taps

In [ ]:
xcen,ycen = taps[0]
buf = 1000  # look at point +/- 1km
ds = NDVI.sel(x=slice(xcen-buf,xcen+buf), y=slice(ycen-buf,ycen+buf))
ds.sel(time=slice('2015-01-01', '2015-06-15')).plot.imshow('x', 'y', col='time', col_wrap=4, cmap='magma', vmin=0, vmax=1)

In [ ]:
ds = NDVI.sel(x=xcen, y=ycen, method='nearest')
timeseries = ds.resample(time='1MS').mean().persist()
s = timeseries.to_series()  #pandas seriers

In [ ]:
# Plot timeseries with HVplot
# Holoviews is also great for interative 2D plots
line = s.hvplot(width=700, height=300, legend=False)
points = s.hvplot.scatter(width=700, height=300, legend=False)
label = f'Mean NDVI: easting={xcen:g} , northing={ycen:g}'

(line * points).relabel(label)